In [131]:
#flu selections

In [2]:
import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)

import os
import matplotlib
import seaborn as sns

from Bio import SeqIO
from Bio.Seq import Seq

import warnings
import itertools
import numpy as np

import tqdm
import gc

import scipy as sp

In [3]:
warnings.simplefilter('ignore')

In [4]:
samples=pd.read_csv('data/samples.csv')
samples

,sample_name,virus_background,replicate,selection,sample_id,Lane,inline_fwd,inline_rev,read_1_file,read_2_file
0,USSR77_R1_Plasmid_L1,USSR77,R1,Plasmid,1,1,TTGATCG,TACCTGA,data/sequencing_data/USSR77_R1_Plasmid_S1_L001...,data/sequencing_data/USSR77_R1_Plasmid_S1_L001...
1,USSR77_R1_Plasmid_L2,USSR77,R1,Plasmid,1,2,TTGATCG,TACCTGA,data/sequencing_data/USSR77_R1_Plasmid_S1_L002...,data/sequencing_data/USSR77_R1_Plasmid_S1_L002...
2,USSR77_R2_Plasmid_L1,USSR77,R2,Plasmid,2,1,CATTCTCGAG,AGAACCAT,data/sequencing_data/USSR77_R2_Plasmid_S2_L001...,data/sequencing_data/USSR77_R2_Plasmid_S2_L001...
3,USSR77_R2_Plasmid_L2,USSR77,R2,Plasmid,2,2,CATTCTCGAG,AGAACCAT,data/sequencing_data/USSR77_R2_Plasmid_S2_L002...,data/sequencing_data/USSR77_R2_Plasmid_S2_L002...
4,USSR77_R1_P1_L1,USSR77,R1,P1,3,1,CCTTAGTACTA,ACCGTTGAC,data/sequencing_data/USSR77_R1_P1_S3_L001_R1_0...,data/sequencing_data/USSR77_R1_P1_S3_L001_R2_0...
...,...,...,...,...,...,...,...,...,...,...
27,SYD21_R2_Plasmid_L2,SYD21,R2,Plasmid,14,2,TTGATCG,AGAACCAT,data/sequencing_data/SYD21_R2_Plasmid_S14_L002...,data/sequencing_data/SYD21_R2_Plasmid_S14_L002...
28,SYD21_R1_P1_L1,SYD21,R1,P1,15,1,CATTCTCGAG,ACCGTTGAC,data/sequencing_data/SYD21_R1_P1_S15_L001_R1_0...,data/sequencing_data/SYD21_R1_P1_S15_L001_R2_0...
29,SYD21_R1_P1_L2,SYD21,R1,P1,15,2,CATTCTCGAG,ACCGTTGAC,data/sequencing_data/SYD21_R1_P1_S15_L002_R1_0...,data/sequencing_data/SYD21_R1_P1_S15_L002_R2_0...
30,SYD21_R2_P1_L1,SYD21,R2,P1,16,1,CCTTAGTACTA,GCAAGCTGTT,data/sequencing_data/SYD21_R2_P1_S16_L001_R1_0...,data/sequencing_data/SYD21_R2_P1_S16_L001_R2_0...


In [450]:
#for idx,row in samples.iterrows():
#    sample_name = row['sample_name']
#    fwd_read = row['read_1_file']
#    rev_read = row['read_2_file']
#    os.system(f'pear -f {fwd_read} -r {rev_read} -o "data/sequencing_data/pear_output/"{sample_name} -j 6')

 ____  _____    _    ____ 
|  _ \| ____|  / \  |  _ \
| |_) |  _|   / _ \ | |_) |
|  __/| |___ / ___ \|  _ <
|_|   |_____/_/   \_\_| \_\

PEAR v0.9.6 [January 15, 2015]

Citation - PEAR: a fast and accurate Illumina Paired-End reAd mergeR
Zhang et al (2014) Bioinformatics 30(5): 614-620 | doi:10.1093/bioinformatics/btt593

Forward reads file.................: data/sequencing_data/USSR77_R1_Plasmid_S1_L001_R1_001.fastq.gz
Reverse reads file.................: data/sequencing_data/USSR77_R1_Plasmid_S1_L001_R2_001.fastq.gz
PHRED..............................: 33
Using empirical frequencies........: YES
Statistical method.................: OES
Maximum assembly length............: 999999
Minimum assembly length............: 50
p-value............................: 0.010000
Quality score threshold (trimming).: 0
Minimum read size after trimming...: 1
Maximal ratio of uncalled bases....: 1.000000
Minimum overlap....................: 10
Scoring method.....................: Scaled score
Threads..

In [135]:
flanks=['AGCCTCCAGTGTAGGATTTGCATTTAA','TGGTGCTTTTGGTCTCCCTGGGG']
all_data = []
for index,row in samples.iterrows():
    gc.collect()
    sample_name = row['sample_name']
    virus_background = row['virus_background']
    replicate = row['replicate']
    selection = row['selection']
    inline_fwd = row['inline_fwd']
    inline_rev = row['inline_rev']
    inline_rev_rcomp = str(Seq(inline_rev).reverse_complement())
    required_length = 78 + 16 + len(inline_fwd) + len(inline_rev)
    barcode_umi_counts = dict()
    for record in tqdm.tqdm(SeqIO.parse(f'data/sequencing_data/pear_output/{sample_name}.assembled.fastq', 'fastq')):
        seq_str = str(record.seq)
        if len(seq_str) == required_length and 'N' not in seq_str and inline_fwd in seq_str[8:8+len(inline_fwd)] and inline_rev_rcomp in seq_str[-8-len(inline_rev_rcomp):-8]:
            UMI_merged = seq_str[:8]+seq_str[-8:]
            if flanks[0] in seq_str and flanks[1] in seq_str:
                barcode = seq_str.split(flanks[0])[1].split(flanks[1])[0]                   
            if flanks[0] in seq_str and flanks[1] not in seq_str:
                barcode = seq_str.split(flanks[0])[1][:28]
            if flanks[0] not in seq_str and flanks[1] in seq_str:
                barcode = seq_str.split(flanks[1])[0][-28:]
                
            barcode_umi_counts.setdefault(barcode, dict())
            barcode_umi_counts[barcode].setdefault(UMI_merged, 0)
            barcode_umi_counts[barcode][UMI_merged] += 1
            
    data = {'barcode':[], 'umi':[], 'count':[]}
    for barcode, umis in barcode_umi_counts.items():
        for umi, count in umis.items():
            data['barcode'].append(barcode)
            data['umi'].append(umi)
            data['count'].append(count)
    data = pd.DataFrame(data)
    #data['sample_name'] = row['sample_name']
    data['sample'] = row['virus_background']+'_'+row['replicate']+'_'+row['selection']
    data['virus_background'] = row['virus_background']
    data['replicate'] = row['replicate']
    data['selection'] = row['selection']
    
    data.to_csv(f'data/barcode_umi_counts/{sample_name}_barcodes_umis.csv')

17891620it [03:25, 87021.77it/s]
18481616it [03:35, 85608.60it/s]
18553528it [03:24, 90775.42it/s]
19202185it [03:30, 91305.03it/s]
27430342it [05:21, 85375.19it/s]
27162240it [05:22, 84264.72it/s]
21998618it [04:18, 84997.17it/s]
21895884it [04:18, 84853.67it/s]
22170553it [04:25, 83592.19it/s]
22151217it [04:24, 83771.33it/s]
21232051it [04:15, 83044.41it/s]
20934808it [04:10, 83421.05it/s]
24040849it [04:35, 87298.99it/s]
23890410it [04:36, 86528.74it/s]
21698923it [04:11, 86225.11it/s]
22509294it [04:17, 87449.80it/s]
13466117it [02:43, 82601.77it/s]
13863049it [02:48, 82236.21it/s]
19027123it [03:50, 82459.52it/s]
19792969it [03:57, 83283.32it/s]
19567286it [03:43, 87559.13it/s]
19511861it [03:43, 87108.52it/s]
19222432it [03:42, 86262.40it/s]
19011350it [03:43, 85098.24it/s]
16992351it [03:27, 81867.88it/s]
17419113it [03:30, 82808.83it/s]
16378556it [03:15, 83800.00it/s]
16537339it [03:17, 83861.53it/s]
20274531it [03:34, 94527.63it/s]
20154534it [03:33, 94332.85it/s]
23639285it

In [136]:
sample_list=[]
for idx,row in samples.iterrows():
    sample=row['sample_name'][:-3]
    if sample in sample_list:
        continue
    if sample not in sample_list:
        sample_list.append(sample)

In [137]:
all_data_df=pd.DataFrame()

for f in tqdm.tqdm(sample_list):
    gc.collect()
    file_lane1 = f+'_L1_barcodes_umis.csv'
    file_lane2 = f+'_L2_barcodes_umis.csv'
    
    if file_lane1 and file_lane2 in os.listdir('data/barcode_umi_counts'):  
        temp_df1 = pd.read_csv(f'data/barcode_umi_counts/{file_lane1}')
        temp_df2 = pd.read_csv(f'data/barcode_umi_counts/{file_lane2}')
        temp_df = pd.concat([temp_df1,temp_df2])
        temp_df['barcode_umi']=temp_df['barcode']+'_'+temp_df['umi']
        temp_df = temp_df.drop_duplicates(subset=['barcode_umi','sample'],keep='first')
        temp_df['bc_count']=1
        temp_df_agg = temp_df.groupby(['virus_background','replicate','selection','sample','barcode']).agg({'bc_count':'sum'}).reset_index()
        all_data_df=pd.concat([all_data_df,temp_df_agg]).reset_index(drop=True)
    
    elif file_lane1 in os.listdir('data/barcode_umi_counts') and file_lane2 not in os.listdir('data/barcode_umi_counts'):  
        temp_df = pd.read_csv(f'data/barcode_umi_counts/{file_lane1}')
        temp_df['bc_count']=1
        temp_df['barcode_umi']=temp_df['barcode']+'_'+temp_df['umi']
        temp_df_agg = temp_df.groupby(['virus_background','replicate','selection','sample','barcode']).agg({'bc_count':'sum'}).reset_index()
        all_data_df=pd.concat([all_data_df,temp_df_agg]).reset_index(drop=True)
        
    elif file_lane2 in os.listdir('data/barcode_umi_counts') and file_lane1 not in os.listdir('data/barcode_umi_counts'):  
        temp_df = pd.read_csv(f'data/barcode_umi_counts/{file_lane2}')
        temp_df['bc_count']=1
        temp_df['barcode_umi']=temp_df['barcode']+'_'+temp_df['umi']
        temp_df_agg = temp_df.groupby(['virus_background','replicate','selection','sample','barcode']).agg({'bc_count':'sum'}).reset_index()
        all_data_df=pd.concat([all_data_df,temp_df_agg]).reset_index(drop=True)
    
    else:
        continue

all_data_df.to_csv('data/all_data_df.csv',index=False)
all_data_df

100%|██████████| 16/16 [17:10<00:00, 64.38s/it]


,virus_background,replicate,selection,sample,barcode,bc_count
0,USSR77,R1,Plasmid,USSR77_R1_Plasmid,AAAACACGCACAAGCTAATACGAAACGA,1
1,USSR77,R1,Plasmid,USSR77_R1_Plasmid,AAAACGACTGCTAGCTCCTAACCGTGGT,1
2,USSR77,R1,Plasmid,USSR77_R1_Plasmid,AAAACTCACTTAAGCTCGGCCCAAGGAT,1
3,USSR77,R1,Plasmid,USSR77_R1_Plasmid,AAAAGGTGTCTTAGCTTGTTCATTATCC,1
4,USSR77,R1,Plasmid,USSR77_R1_Plasmid,AAAAGTTGACATAGCTGGTCAATATTAC,1
...,...,...,...,...,...,...
10193604,SYD21,R2,P1,SYD21_R2_P1,TTTTGCTAGGGTAGCTTGTATCACGTGG,1
10193605,SYD21,R2,P1,SYD21_R2_P1,TTTTGGGACTACAGCTATCTCATCTCAG,1
10193606,SYD21,R2,P1,SYD21_R2_P1,TTTTTGACCTCATGCTACCTTGTATTCA,1
10193607,SYD21,R2,P1,SYD21_R2_P1,TTTTTTATTGTTAGCTTATAAACTGGTA,1
